we will read the data from the file in my Drive. 

will narrow it to 1000 documents that are relatively short and similar

save them to a df with the original index

download the df as csv to use in next notebooks

In [1]:
# Imports
import sklearn
import pandas as pd
import numpy as np
import random
import re
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
from itertools import islice
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

loading the data from colab

In [2]:
# Uploading the data
COLAB = True

In [3]:
if COLAB:
    from google.colab import drive
    from os.path import exists

    amazon_train_file = 'train.ft.txt.bz2'

    if exists(amazon_train_file):
        train_file = amazon_train_file
    else:
        drive.mount('/content/drive/')

        # For Hadas' drive
        #my_dir = 'drive/MyDrive/Y-data/Intuit-K-anonimity/'

        # For Lior's drive
        my_dir = 'drive/MyDrive/Y-data/Y-DATA_PROJECT/'

        train_file = my_dir + '/train.ft.txt.bz2'
else:
    train_file = '../data/' + 'train.ft.txt.bz2'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# Credit https://www.kaggle.com/code/anshulrai/cudnnlstm-implementation-93-7-accuracy

import bz2

# Readling the file to list of comments
train_file = bz2.BZ2File(train_file)
train_file_lines = train_file.readlines()

# Converting from raw binary strings to strings that can be parsed
train_file_lines = [x.decode('utf-8') for x in train_file_lines]

# Extracting the labels and sentences
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines] # And converting to lower case

In [5]:
del(train_file_lines)  # Free RAM

In [6]:
# create a data frame from train data:
df = pd.DataFrame(list(zip(train_sentences, train_labels)), columns =['txt', 'sentiment'])
df.head()

,txt,sentiment
0,stuning even for the non-gamer: this sound tra...,1
1,the best soundtrack ever to anything.: i'm rea...,1
2,amazing!: this soundtrack is my favorite music...,1
3,excellent soundtrack: i truly like this soundt...,1
4,"remember, pull your jaw off the floor after he...",1


filter only docs with less than 30 words/tokens

In [7]:
df['num_of_words'] = df['txt'].apply(lambda x : len(x.split(' ')))

In [8]:
max_len = 28
df_short_sentences = df[df['num_of_words'] <= max_len] # Filtering using pandas is much faster
print(df_short_sentences.shape)

(363829, 3)


In [9]:
# Free RAM
del(df)

calc the jaccard to filter more similar sentences

In [15]:
def jaccard_index(sentence1, sentence2):
    """ Calc Jaccard index for each pair of sentences """
    words1 = set(sentence1.split())
    words2 = set(sentence2.split())
    intersection = len(words1.intersection(words2))
    union = len(words1.union(words2))
    jaccard = intersection / union
    return jaccard

In [10]:
indices_list = list(df_short_sentences.index)
len(indices_list)

363829

In [11]:
indices_list_short_1 = indices_list[1:4000] # run for shorter time

In [16]:
%%time
# Create a list of sentence texts
sentences = list(df_short_sentences.txt.loc[indices_list_short_1])

# Compute the Jaccard index for all pairs of sentences
jaccard_index_dict = {(indices_list_short_1[i],indices_list_short_1[j]): jaccard_index(sentences[i], sentences[j]) for i in range(len(sentences)) for j in range(i+1, len(sentences))}

# Sort the dictionary by its values in descending order
sorted_dict = dict(sorted(jaccard_index_dict.items(), key=lambda item: item[1], reverse=True))

CPU times: user 1min 40s, sys: 1.7 s, total: 1min 41s
Wall time: 1min 42s


In [ ]:
def plot_jaccard_hist(sorted_dict):
  '''creat a hist of jaccard scores'''
  # extract the values from the dictionary
  dic_values = list(sorted_dict.values())

  # plot a histogram of the values
  plt.hist(dic_values, bins=50, range = (0,0.3))

  # set labels and title
  plt.xlabel('Values')
  plt.ylabel('Counts')
  plt.title('Histogram of Jaccard Indecs')

  # display the plot
  plt.show()

In [ ]:
plot_jaccard_hist(sorted_dict)

In [17]:
dic_values = list(sorted_dict.values())
print("mean Jaccard:", np.mean(dic_values))

mean Jaccard: 0.05923649419314845


In [54]:
# Getting the N documents with nearest neighbors
n = 1722
first_n_items = dict(islice(sorted_dict.items(), n)) # work with n sentences as a start
# Get the list of indexes
unique_numbers = [num for tup in first_n_items.keys() for num in tup]  # list of indeces for sentences to work with


In [55]:
# Free RAM
del(first_n_items)

In [56]:
# Filter the df to recive only these sentences and print df
filtered_df = df_short_sentences.loc[df_short_sentences.index.isin(unique_numbers)]
print(filtered_df.head())

                                                   txt  sentiment  \
84   voices from the farm: i have a copy of this bo...          1   
99   caution!: these tracks are not the "original" ...          0   
100  textbook: book shipped quickly and was in exce...          1   
243  it was great!!!: i like it. it was very cool t...          1   
269  dockers do the job.: this product was just wha...          1   

     num_of_words  
84             27  
99             22  
100            16  
243            21  
269            25  


save the df to a csv


In [60]:
filtered_df.to_csv('df_filtered.csv', index=True)


In [51]:
filtered_df.head()

,txt,sentiment,num_of_words
84,voices from the farm: i have a copy of this bo...,1,27
99,"caution!: these tracks are not the ""original"" ...",0,22
100,textbook: book shipped quickly and was in exce...,1,16
243,it was great!!!: i like it. it was very cool t...,1,21
269,dockers do the job.: this product was just wha...,1,25


In [58]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1007 entries, 84 to 41255
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   txt           1007 non-null   object
 1   sentiment     1007 non-null   int64 
 2   num_of_words  1007 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 31.5+ KB
